In [ ]:
import uproot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import rcParams
import seaborn as sns
import scienceplots
sns.set(font_scale=1.0)
sns.set_style('white')
from IPython.display import Image


import matplotlib.font_manager as font_manager

#legend_properties = {'weight':'bold'}

#plt.style.use('fivethirtyeight') # fivethirtyeight is name of styl
#plt.style.use(['science','no-latex'])
#comparing the result in this presentation slide 4
#https://docs.google.com/presentation/d/14KgpLBpJwNQH5tjEczl9tPkInSiKkJtjgqBPWSyoXok/edit#slide=id.g1b79411b324_0_80

In [ ]:
rcParams.update({'figure.autolayout':True})
rcParams.update({'figure.figsize':[12,8]})
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['lines.linewidth'] = 2.5
plt.rcParams['grid.linewidth'] = 2.5
plt.rcParams['grid.linestyle']=':'

# HELPER FUNCTION

In [ ]:
def my_plotter(ax,ene,weight,label_value,yscale,xlabel='Energy [keV]',ylabel='Counts',iso="Th228"):
    """
    A helper function to make a graph

    Parameters
    ----------
    ax : Axes
        The axes to draw to

    data1 : array
       The x data

    data2 : array
       The y data

    param_dict : dict
       Dictionary of keyword arguments to pass to ax.plot

    Returns
    -------
    out : list
        list of artists added
    """
    out=ax.hist(ene, bins=200, histtype=u'step', weights=weight, density=True, label=label_value)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_yscale(yscale)
    ax.set_title(f"Energy PDF for SS events  for {iso}")
    ax.legend()
    
    return out

In [ ]:
#input the root file and ouput the filtered dataframe

def get_filtered(file,apply_filter,use_filter=False):
    '''
    returns the pandas dataframe for the given root file using the uproot library. If use_filter=True, indicated filter will be used.
    '''
    f=uproot.open(file+':tree')
    df=f.arrays(f.keys(),library='pd')
    if use_filter:return df.query(apply_filter).reset_index(drop=True)
    return df#.query('energy<3000').reset_index(drop=True)
    
    


In [ ]:
# returns the plot for the isotopes in the given dataframe 
def plot_isotopes(df,df_label="PX",scale_kind="log"):
    '''function to plot the isotopes based on the isotope number'''
    #isotopes=df.isotope.unique()
    ra224_df=df.query('isotope==1')
    pb212_df=df.query('isotope==2')
    bi212_df=df.query('isotope==3')
    tl208_df=df.query('isotope==4')
    
    ra224_energy,ra224_weight=ra224_df.energy,ra224_df.weight
    pb212_energy,pb212_weight=pb212_df.energy,pb212_df.weight
    bi212_energy,bi212_weight=bi212_df.energy,bi212_df.weight
    tl208_energy,tl208_weight=tl208_df.energy,tl208_df.weight
    
    fig, ax = plt.subplots(1, 1,figsize=(6,4))
    my_plotter(ax,ra224_energy,ra224_weight,df_label+" Ra224",scale_kind)
    my_plotter(ax,pb212_energy,pb212_weight,df_label+" Pb212",scale_kind)
    my_plotter(ax,bi212_energy,bi212_weight,df_label+" Bi212",scale_kind)
    my_plotter(ax,tl208_energy,tl208_weight,df_label+" Tl208",scale_kind)
    plt.show()
    

In [ ]:
def display_fractions(df,cut,total_simulations,greek_name,use_filter=True):
    '''
    displays the fraction in the cut dataframe w.r.t. total simulations
    '''
    df_temp=get_filtered(df,cut,use_filter=use_filter)
    df_count=df_temp.shape[0]
    greek=df_count/total_simulations
    print(30*'--')
    print(f"""
    Shape of  dataframe                    : {df_temp.shape}
    Number of rows (i.e. events count)     : {df_count}
    {greek_name}                           : {greek}
    {greek_name} %                         : {greek:.2%}

    """)
    print(30*'--')
    return greek

In [ ]:
def print_isotopes(df,name,isotopes):
    '''
    reuturns the isotopes information for given df
    '''
    list_iso=df.isotope.unique()
    non_iso=df.isotope.value_counts().to_list()[::-1]
    #isotopes=['Th228','Ra224','Rn220','Po216','Pb212','Bi212','Tl208']
    #isotope coutns before the cut
    counts_df=pd.DataFrame(df.isotope.value_counts().reset_index().values,columns=['isonum','counts'])\
    .sort_values(by=['isonum'])\
    .reset_index(drop=True)

    counts_df["isotopes"]=counts_df.isonum.apply(lambda x:isotopes[x-1])
    counts_df=counts_df[['isonum','isotopes','counts']]
    # #raw_counts.index=raw_counts.isotopes
    #print(raw_counts.to_string())
    #print(f"isotopes counts for {name}:\n\n{counts_df.to_string()}\n")
    return counts_df


In [ ]:
# bar graph plot
def get_bar_plot(df,title="test"):
    #df_deposited=df[['isotopes','counts']]
    skipEThreshold=0.1
    plt.figure(figsize=(6,4))
    if df.empty:print(f"{df} is empty !")
    ax=df.plot.bar(x='isotopes',y='counts',title=title)
    for c in ax.containers:
        # set the bar label
        ax.bar_label(c, fmt='%.0f', label_type='edge',rotation=30)
    plt.xlabel('ISOTOPES')
    #plt.title(f'Isotope counts in g4tree for 1M Th228 at all positions for skipEThreshold: {skipEThreshold} keV')
    plt.ylabel('COUNTS')
    plt.yscale('log')
    plt.savefig(f"{title}.pdf")
    #plt.savefig(f'g4tree_isotopes_bothskipEThreshold: {skipEThreshold}.pdf')
    plt.show()

In [ ]:
#produces the energy spectrum
def get_energy_spectrum(df,loc,iso,isotopes,bins=1000,weights="weights"):
    '''returns the energy spectrum including the weights of isotopes'''
    
    sns.set(rc={'figure.figsize':(20,10)})
    
    #adding the isotop name column in the df dataframe
    df=df[df.energy<3000]
    iso_list=[i for i in range(1,len(isotopes))]
    iso_dict=dict(zip(iso_list,isotopes))
    df['iso_name']=df['isotope'].map(iso_dict)
    
    
    #sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
    g=sns.histplot(data=df, x="energy",hue="iso_name",weights="weight",bins=bins,element="step",fill=False)#,palette=['r','b','g','y','k'])
    #plt.legend(labels=iso_names)
    g.set_xlabel('Energy [keV]')
    g.set_ylabel('Counts')
    plt.yscale("log")
    plt.title(f'Energy specturm pdf for {iso} at {loc}')
    plt.show()

In [ ]:
def get_spec(df,bins=2000,iso='Co60',loc='TPC',isotopes=['Co60']):
    '''returns the energy spectrum including the weights of isotopes'''
    
    sns.set(rc={'figure.figsize':(20,10)})
    df_temp=df[df.energy<3000]
    iso_dict=dict(zip(iso_list,isotopes))
    df_temp['iso_name']=df_temp['isotope'].map(iso_dict)


    #sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
    g=sns.histplot(data=df_temp, x="energy",hue="iso_name",weights="weight",bins=bins,element="step",fill=False)#,palette=['r','b','g','y','k'])
    #plt.legend(labels=iso_names)
    g.set_xlabel('Energy [keV]')
    g.set_ylabel('Counts')
    plt.yscale("log")
    plt.title(f'Energy specturm pdf for {iso} at {loc}')
    plt.savefig(f"{iso}-{loc}.pdf")
    plt.show()

In [ ]:
isotopes=['Ra226','Rn222','Po218','Pb214','Bi214','Po214','Pb210','Bi210','Po210','Po206']
iso_list=[i for i in range(1,len(isotopes))]
iso_dict=dict(zip(iso_list,isotopes))
iso_dict

# Co60

In [ ]:
# 1173.228 3 	     99.85 % 3 	  1.1715 4 
# 1332.492 4 	     99.9826 % 6 	  1.332260 9
Image(filename='Co60.png') 
Image(filename='co60rate.jpg') 


In [ ]:
# 1173.228 3 	     99.85 % 3 	  1.1715 4 
# 1332.492 4 	     99.9826 % 6 	  1.332260 9
Image(filename='Co60.png') 



In [ ]:
Image(filename='srcpos.png',width=500) 

# ROOT DATA FILES

- s12 (intensity at PZ and NZ one-tenth of other positions)
#- s15 (intensity same at all positions)


In [ ]:
#give the name of rootfile

file='/home/thakur/slac_data/s12/s12_Co60_all.root'

isotope='Co60'
isotopes=['Co60']

In [ ]:

Image(filename='imagesteps.png') 

- $\alpha = \frac{\text{number in g4tree}}{\text{total simulations}} $


- $\beta = \frac{\text{number in inner 1 tonne}}{\text{total simulations}} $
- $\gamma = \frac{\text{SS peak events in inner 1 tonne}}{\text{total simulations}} $

# FILTERS

In [ ]:

#this is beta filter
#inner 1 tonne, ms, ss

beta_filter='''
(standoff>201.086) &\
passed_xy_thresh &\
passed_z_thresh &\
(n_x_ch_abovenoise > 0) &\
(n_y_ch_abovenoise > 0) &\
(m_nOPCal < (1.077 * m_nQ + 313)) &\
(m_nOPCal > (0.597 * m_nQ - 216)) &\
~NESTBugFound &\
~NearAnodeBugFound
'''


#this is gamma filter inner 1 tonne, ss, peak events (2610 to 2620)
gamma_filter1='''
(standoff>201.086) &\
m_DNNvalue>0.85&\
(abs(energy-1173)<10)&\
passed_xy_thresh &\
passed_z_thresh &\
(n_x_ch_abovenoise > 0) &\
(n_y_ch_abovenoise > 0) &\
(m_nOPCal < (1.077 * m_nQ + 313)) &\
(m_nOPCal > (0.597 * m_nQ - 216)) &\
~NESTBugFound &\
~NearAnodeBugFound
'''

gamma_filter2='''
(standoff>201.086) &\
m_DNNvalue>0.85&\
(abs(energy-1332)<10)&\
passed_xy_thresh &\
passed_z_thresh &\
(n_x_ch_abovenoise > 0) &\
(n_y_ch_abovenoise > 0) &\
(m_nOPCal < (1.077 * m_nQ + 313)) &\
(m_nOPCal > (0.597 * m_nQ - 216)) &\
~NESTBugFound &\
~NearAnodeBugFound
'''

dec102020_filter='''
(standoff>100) &\
passed_xy_thresh &\
passed_z_thresh &\
(n_x_ch_abovenoise > 0) &\
(n_y_ch_abovenoise > 0) &\ 
(m_nOPCal < (1.077 * m_nQ + 313)) &\
(m_nOPCal > (0.597 * m_nQ - 216)) &\
~NESTBugFound &\
~NearAnodeBugFound &\
m_DNNvalue>0.85'''




jason_filter='''
(standoff>100) &\
m_DNNvalue>0.85 &\
passed_xy_thresh &\
passed_z_thresh &\
(n_x_ch_abovenoise > 0) &\
(n_y_ch_abovenoise > 0) &\
(m_nOPCal < (1.077 * m_nQ + 313)) &\
(m_nOPCal > (0.597 * m_nQ - 216))&\
(abs(energy-2614)<10)
'''
#~NESTBugFound &\
#~NearAnodeBugFound &\
# & ~NESTBugFound & ~NearAnodeBugFound' & (abs(energy-2614)<10)'''


In [ ]:
# beta filter
print(20*'==')
print("beta filter:\n", beta_filter.replace('&','\n'))
print(20*'==')
print("gamma1 filter:\n", gamma_filter1.replace('&','\n'))
print(20*'==')
print("gamma2 filter:\n", gamma_filter2.replace('&','\n'))
print(20*'==')
# print("jason filter:\n", jason_filter.replace('&','\n'))
# print(20*'==')

# Total Simulations

In [ ]:
# Total simulations
total_simulations= 1e7
file_name=file

print(f"Total Simulations: {total_simulations:0.1e}")


# $\alpha$ (DEPOSITS IN TPC)

In [ ]:
df_alpha=get_filtered(file_name,None,use_filter=False)
g4tree_count=df_alpha.shape[0]
raw_alpha=g4tree_count/total_simulations

print(f"""
Shape of  dataframe                        : {df_alpha.shape}
Number of rows (total events in g4tree)    : {g4tree_count:e}
raw_alpha                                  : {raw_alpha}
raw_alpha %                                : {raw_alpha:.3%}

""")

In [ ]:
df_alpha

In [ ]:
# bins=2000
# iso='Co60'
# loc='TPC'
# df_alpha=df_alpha[df_alpha.energy<3000]
# iso_dict=dict(zip(iso_list,isotopes))
# df_alpha['iso_name']=df_alpha['isotope'].map(iso_dict)


# #sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
# g=sns.histplot(data=df_alpha, x="energy",hue="iso_name",weights="weight",bins=bins,element="step",fill=False)#,palette=['r','b','g','y','k'])
# #plt.legend(labels=iso_names)
# g.set_xlabel('Energy [keV]')
# g.set_ylabel('Counts')
# plt.yscale("log")
# plt.title(f'Energy specturm pdf for {iso} at {loc}')
# plt.show()

In [ ]:
get_spec(df_alpha)

In [ ]:
#get_energy_spectrum(df_alpha,isotopes=isotopes,iso='co60',loc='TPC')

In [ ]:
df_alpha.isotope.value_counts()  #isotopes

In [ ]:
alpha_df=print_isotopes(df_alpha,"df_alpha",isotopes=isotopes)
alpha_df

In [ ]:
get_bar_plot(print_isotopes(df_alpha,"df_alpha",isotopes=isotopes),f"Deposits in TPC for {total_simulations:0.1e} {isotope} primary events at all 6 positions")


In [ ]:
# list_iso=df_alpha.isotope.unique()
# non_iso=df_alpha.isotope.value_counts().to_list()[::-1]
# isotopes=['Th228','Ra224','Rn220','Po216','Pb212','Bi212','Tl208']
# #isotope coutns before the cut
# raw_counts_df=pd.DataFrame(df_alpha.isotope.value_counts().reset_index().values,columns=['isonum','raw_counts'])\
# .sort_values(by=['isonum'])\
# .reset_index(drop=True)

# raw_counts_df["isotopes"]=raw_counts_df.isonum.apply(lambda x:isotopes[x-1])
# raw_counts_df=raw_counts_df[['isonum','isotopes','raw_counts']]
# # #raw_counts.index=raw_counts.isotopes
# #print(raw_counts.to_string())
# raw_counts_df


# $\beta$ (DEPOSITS IN INNER ONE TONNE)

In [ ]:
df_beta=get_filtered(file_name,beta_filter,use_filter=True)
raw_beta_count=df_beta.shape[0]
raw_beta=raw_beta_count/total_simulations

print(f"""
Shape of  dataframe                        : {df_beta.shape}
Number of rows                             : {raw_beta_count:e}
raw_beta                                   : {raw_beta_count}
raw_beta  %                                : {raw_beta:.3%}

""")

In [ ]:
#get_energy_spectrum(df_beta,iso='Co60',isotopes=isotopes,loc='INNER 1 TON')

In [ ]:
get_spec(df_beta,iso='Co60',isotopes=isotopes,loc='INNER 1 TON')

In [ ]:
beta_df=print_isotopes(df_beta,"df_beta",isotopes=isotopes)
beta_df

In [ ]:
get_bar_plot(print_isotopes(df_beta,"df_beta",isotopes=isotopes),f"Deep trigger deposits for {total_simulations:0.1e} {isotope} primary events")


# SS EVENTS IN INNER ONE TONNE

In [ ]:
df_ss=df_beta.query('m_DNNvalue>0.85')
df_ss

In [ ]:
#df_beta=get_filtered(file_name,beta_filter,use_filter=True)
raw_ss_count=df_ss.shape[0]
raw_ss=raw_ss_count/total_simulations

print(f"""
Shape of  dataframe                        : {df_ss.shape}
Number of rows                             : {raw_ss_count:e}
raw_ss                                     : {raw_ss_count}
raw_ss %                                   : {raw_ss:.3%}

""")

In [ ]:
get_bar_plot(print_isotopes(df_ss,"df_ss",isotopes=isotopes),f"SS inner 1 tonne deposits for {total_simulations:0.1e} {isotope} primary events")


In [ ]:
get_spec(df_ss,iso='Co60',isotopes=isotopes,loc='SS INNER 1 TON')

# $\gamma$ (SS, PEAKS EVENTS IN INNER ONE TONNE)

# 1163 < Energy < 1183

In [ ]:
df_gamma=get_filtered(file_name,gamma_filter1,use_filter=True)
raw_gamma_count=df_gamma.shape[0]
raw_gamma=raw_gamma_count/total_simulations

print(f"""
Shape of  dataframe                        : {df_gamma.shape}
Number of rows                             : {raw_gamma_count:e}
raw_gamma                                  : {raw_gamma_count}
raw_gamma %                                : {raw_gamma:.5%}

""")

In [ ]:
df_gamma.energy

In [ ]:
gamma_df=print_isotopes(df_gamma,"df_gamma",isotopes=isotopes)
gamma_df

In [ ]:
#if not gamma_df.empty:get_bar_plot(print_isotopes(df_gamma,"df_gamma"),f"Deep trigger peak deposits for {total_simulations:0.1e} {isotope} primary events at all positions")

In [ ]:
get_spec(df_gamma,iso='Co60',isotopes=isotopes,loc='INNER 1 TON, SS, PEAK (1173 keV)')

# 1322 < Energy < 1342

In [ ]:
df_gamma=get_filtered(file_name,gamma_filter2,use_filter=True) #gamma2 filter
raw_gamma_count=df_gamma.shape[0]
raw_gamma=raw_gamma_count/total_simulations

print(f"""
Shape of  dataframe                        : {df_gamma.shape}
Number of rows                             : {raw_gamma_count:e}
raw_gamma                                  : {raw_gamma_count}
raw_gamma %                                : {raw_gamma:.5%}

""")

In [ ]:
df_gamma.energy

In [ ]:
gamma_df=print_isotopes(df_gamma,"df_gamma",isotopes=isotopes)
gamma_df

In [ ]:
get_spec(df_gamma,iso='Co60',isotopes=isotopes,loc='INNER 1 TON, SS, PEAK (1332 keV)')

In [ ]:
alpha_list=alpha_df['counts'];beta_list=beta_df['counts'];gamma_list=gamma_df['counts'];index_name=alpha_df['isotopes']

alpha_list,beta_list,gamma_list,index_name


In [ ]:
index_name

In [ ]:
alpha_beta_gamma=pd.DataFrame({"isotopes":index_name,"tpc_deposits":alpha_list})#,"ss_deep_deposits":beta_list})#,"tpc_deposits":alpha_list,"ss_deep_deposits":beta_list,"ss_deep_peak_deposits":gamma_list})#,index=index_name)
# alpha_beta_gamma['
# alpha_beta_gamma
alpha_beta_gamma

In [ ]:
col_name=alpha_df.columns[1:]
col_name

In [ ]:
#net_df=pd.concat([alpha_df[col_name].rename(columns={"counts":"tpc_deposits"}),beta_df[col_name].rename(columns={"counts":"deep_ss_deposits"}),gamma_df[col_name].rename(columns={"counts":"deep_ss_peak_deposits"})])
#net_df

# ISOTOPES INFORMATION

In [ ]:
# iso_br=[1]#,1,1,1,1,0.3594] #branching ratios of Th228 to other isotopes

# combined_counts_df=alpha_df.rename(columns={'counts':'deposited'}).merge(gamma_df.rename(columns={'counts':'good'}),how='left').fillna(0)
# combined_counts_df.insert(loc=2,column="sim_events",value=6*[total_simulations])
# combined_counts_df['alpha']=combined_counts_df['deposited']/combined_counts_df['sim_events']
# combined_counts_df['gamma']=combined_counts_df['good']/combined_counts_df['sim_events']
# combined_counts_df['branching_ratio']=iso_br
# raw_br=sum([a*b for a,b in zip(combined_counts_df.deposited,combined_counts_df.branching_ratio)])
# good_br=sum([a*b for a,b in zip(combined_counts_df.good,combined_counts_df.branching_ratio)])
# alpha_br=sum([a*b for a,b in zip(combined_counts_df.alpha,combined_counts_df.branching_ratio)])
# gamma_br=sum([a*b for a,b in zip(combined_counts_df.gamma,combined_counts_df.branching_ratio)])

# # #adding a row
# combined_counts_df.loc[len(combined_counts_df.index)]=['-','Th228_Chain',total_simulations,raw_br,good_br,alpha_br,gamma_br,1]
# combined_counts_df



# comparable results to:
[jason's result](https://docs.google.com/presentation/d/14KgpLBpJwNQH5tjEczl9tPkInSiKkJtjgqBPWSyoXok/edit#slide=id.g1b79411b324_0_9])

In [ ]:
# alpha and gamma including the branching ratios
alpha=raw_alpha;gamma=raw_gamma;beta=raw_beta

In [ ]:
print(f"""
alpha  : {alpha}
beta   : {beta}
gamma  : {gamma}
""")

In [ ]:

#get_energy_spectrum(df_alpha,'TPC')
#df_alpha.energy.plot()

In [ ]:

#get_energy_spectrum(df_beta,'INNER 1 TON')
#df_alpha.energy.plot()

In [ ]:

#get_energy_spectrum(df_gamma,'INNER 1 TON (SS, PEAK)')


In [ ]:
# iso=['Th228','Ra224','Rn220','Pb212','Bi212','Tl208']
# col=['r','y','b','k','c','m']
# for i in range(1,len(iso)+1):
#     #print(non_filtered_th228_px.head())
#     df_new=df_alpha[df_alpha.isotope==i]
#     if df_new.empty:continue
#     #print(df_new.head())
#     px_energy=df_new.energy;
#     plt.hist(px_energy,200,density=False,histtype='step',weights=df_new.weight,alpha=0.95,label=iso[i-1],color=col[i-1])
#     plt.yscale('log')
#     plt.xlabel('Energy [keV]')
#     plt.ylabel('G4tree counts ')
#     plt.legend()
# plt.title("Deposited Counts vs Energy for Th228 at all positions (B.R. consideration)")
# plt.show()

In [ ]:
#df_alpha['energy'].plot()

In [ ]:
# combined_counts_df.loc[len(combined_counts_df)]=new_row
# combined_counts_df

In [ ]:
Image(filename='rate.png') 

In [ ]:
#time
t=10e-3 # 10 ms in the simulation 

print(f"Drift time: {t} s")
#t=1e-3

In [ ]:
#jason's values
#alpha=0.0767;gamma=3.3e-5;t=1e-3
#alpha=4.63e-1;gamma=1.99e-5;t=1e-3 #raymond
#alpha=0.069;gamma=1.29e-7;t=1e-3

In [ ]:
# alpha and gamma
print(f"""
alpha  : {alpha}
beta   : {beta}
gamma  : {gamma}
time   : {t} s
""")

# $A_{\text{optimal}}=\frac{1+\sqrt{5}}{2\alpha t} $ 
# ${\text{t}}=\frac{1+\sqrt{5}}{2\alpha A_{\text{optimal}}} $ 

In [ ]:
#plotting activity vs rate and activity vs time in the same graph
#source activity
#sns.set_style('whitegrid')
sns.set(style='ticks')
# source_activity=3750     #850x4+85x2
# deployment_time=2*60*60  #2 hrs
#txt=str(source_activity)+" Bq"
x_range_max=2500
x=np.linspace(1,x_range_max,5000)                              #defination of x

y_best=np.array([gamma*A*np.exp(-beta*A*t) for A in x])                        #best
y_realistic=np.array([gamma*A*(1+alpha*A*t)*np.exp(-alpha*A*t) for A in x])    #realistic
y_worst=np.array([gamma*A*np.exp(-alpha*A*t) for A in x])                      #worst


#time sequence for N events
N_events=1000
to_hrs=60*60
t_best=N_events/(y_best*to_hrs)
t_realistic=N_events/(y_realistic*to_hrs)
t_worst=N_events/(y_worst*to_hrs)

arg_y_max=np.argmax(y_realistic)
arg_y_min=np.argmin(y_realistic)

y_max=y_realistic[arg_y_max]
y_min=y_realistic[arg_y_min]

A_max=x[arg_y_max]

#time for A_optimal
#t_min=(1+np.sqrt(5))/(2*alpha*A_max)
t_min=N_events/y_max

# # print(f"A_min                   : {A_min:0.2f}")
# # print(f"A_max_index             : {arg_max:0.2f}")
# # print(f"A_max                   : {y_max:0.2f}")
# # print(f"Min_rate                : {y[arg_y_min]:0.2f} Hz ({y[arg_max]*60:0.2f} Evetns/min)")
# #print(f"Activity for Max Rate   : {x[arg_max]:0.2f} kBq\n")
print(f"Max_rate                : {y_max:0.5f} Hz ({y_max*60:0.5f} Events/min)")
print(f"Min_time                : {t_min:0.5f} s or {t_min/60:0.5f} mins or {t_min/(60*60):0.5f} hrs")
print(f"Activity for Max Rate   : {A_max:0.5f} Bq\n")

#labels
label_best=r' [ $\gamma A e^{-\beta A t}$ ]'
label_real=r' [ $\gamma A (1+\alpha A t)e^{-\alpha A t}$ ]'
label_worst=r' [ $\gamma A e^{-\alpha A t}$ ]'
colors=['b','g','r']

fig,axs=plt.subplots(2,figsize=(12,12))
font = font_manager.FontProperties(family='monospace',
                                   weight='bold',
                                   style='normal', size=15)

font1 = {'family':'monospace','color':'k','size':15} #font for labels

legends=[fr'best',fr'realistic',fr'worst']
axs[0].plot(x,y_best,colors[0],label=legends[0]+label_best)
axs[0].plot(x,y_realistic,colors[1],label=legends[1]+label_real)
axs[0].plot(x,y_worst,colors[2],label=legends[2]+label_worst)
#axs[0].axvline(x=source_activity,color='m',linestyle='--')
#axs[0].text(source_activity+150,0.3,txt,fontsize=20,rotation='vertical')
# axs[0].plot(x,y2,'r',label=legends[1])
# axs[0].plot(x,y3,'pink',label=legends[2])
axs[0].set_xlabel('Activity [Bq]',fontdict=font1)
axs[0].set_ylabel('Rate [Hz]',fontdict=font1)
axs[0].set_title(f'Rate of accumulation of SS Deep Peak Events for {isotope}',fontdict=font1)
axs[0].grid()
axs[0].tick_params(direction='out', 
                   labelsize=15,
                   #length=6, 
                   #width=3, 
                   colors='k',
               grid_color='k', 
                   grid_alpha=0.75,
                  grid_linestyle=':')
axs[0].legend(prop=font)

#time

# axs[1].plot(x,1/y_best*1/60,colors[0],label=legends[0])
# axs[1].plot(x,1/y_realistic*1/60,colors[1],label=legends[1])
# axs[1].plot(x,1/y_worst*1/60,colors[2],label=legends[2])

axs[1].plot(x,t_best,colors[0],label=legends[0])
axs[1].plot(x,t_realistic,colors[1],label=legends[1])
axs[1].plot(x,t_worst,colors[2],label=legends[2])
#axs[1].axvline(x=source_activity,color='m',linestyle='--')
#axs[1].text(source_activity+10,10,txt,fontsize=20)
# axs[1].plot(x,1/y2*1/60,'r',label=legends[1])
# axs[1].plot(x,1/y3*1/60,'pink',label=legends[2])
axs[1].set_xlabel('Activity [Bq]',fontdict=font1)
axs[1].set_ylabel('Time [hr]',fontdict=font1)
axs[1].set_title(f'Time to accumulate {N_events} SS Deep Peak Events for {isotope}',fontdict=font1)
axs[1].set_yscale('log')
axs[1].grid()
axs[1].tick_params(direction='out', 
                   labelsize=15,
                   #length=6, 
                   #width=3, 
                   colors='k',
               grid_color='k', 
                   grid_alpha=0.75,
                  grid_linestyle=':')
axs[1].legend(prop=font)
fig.tight_layout(pad=10.0)
#deployment time vs count for realistic
# dep_time=np.arange(deployment_time+500)
# N_best=np.array([(gamma*source_activity*np.exp(-beta*source_activity*t))*time for time in dep_time])  
# N_realistic=np.array([(gamma*source_activity*(1+alpha*source_activity*t)*np.exp(-alpha*source_activity*t))*time for time in dep_time])  
# N_worst=np.array([(gamma*source_activity*np.exp(-alpha*source_activity*t))*time for time in dep_time])
# axs[2].plot(dep_time,N_best,colors[0],label=legends[0])
# axs[2].plot(dep_time,N_realistic,colors[1],label=legends[1])
# axs[2].plot(dep_time,N_worst,colors[2],label=legends[2])
# axs[2].axvline(x=deployment_time,color='b',linestyle=':')

# axs[2].set_xlabel('Deployment time [s]')
# axs[2].set_ylabel('Expected Counts')
# axs[2].text(deployment_time+20,1000,'2 hrs',rotation='vertical',fontsize=20)
# axs[2].grid()
# axs[2].legend()
save_fig=True
if save_fig:plt.savefig(f'{isotope}rateestimateion.pdf',dpi=600)

plt.show()

#print(t_best)

In [ ]:
#optimal rates for different circumstances
arg_y_max_realistic=np.argmax(y_realistic)
#arg_y_min=np.argmin(y_realistic)

# y_max_realistic=y_realistic[arg_y_max_realistic]
# #y_min=y_realistic[arg_y_min]

# A_max=x[y_max_realistic]
# A_max
arg_y_max_realistic

y_realistic[arg_y_max_realistic],x[arg_y_max_realistic]

In [ ]:
#optimal rates for different circumstances
def get_y_max_x_opt(kind,y,N=1000):
    '''
    returns the maximum value of the rate as well as corresponding optimal activity of a `kind' of rate
    '''
    arg_y_max=np.argmax(y)

    return [kind,y[arg_y_max],x[arg_y_max],N/(60*60*y[arg_y_max])]

In [ ]:
R_best=get_y_max_x_opt('best',y_best)                      #best_rate
R_realistic=get_y_max_x_opt('realistic',y_realistic)       #ralistic_rate
R_worst=get_y_max_x_opt('worst',y_worst)                   #worst_rate
df_summary=pd.DataFrame([R_best,R_realistic,R_worst],columns=['case','max_rate(Hz)','optimal_activity(Bq)','1000_events_time[hr]'])
df_summary['2hrs_counts']=2000/df_summary['1000_events_time[hr]']
df_summary.round(2)

In [ ]:
reall=R_realistic[2]
reall

In [ ]:
for i,j in enumerate(range(1,1000),start=1):
    total=j*4+(j*2)/10
    if total>(reall-20):print(f"{i}  =>    {total}")
    if total>(reall+20):break
    

In [ ]:
#usable deep events with realistic approach
# A=source_activity
# N=deployment_time*gamma*A*(1+alpha*A*t)*np.exp(-alpha*A*t)
# print(f"Counts with 2 hrs deployment {N:0.2f}")
A_opt=200
A_opt*4+(A_opt/10)*2


# Electron Lifetime Determination
- slice the data into 20 bins along z-axis
- fit 2.6 MeV peak in each z-slice
- fitted peaks are fitted with an exponential function to get electron lifetime ($\tau$)

In [ ]:
#raw data
df_alpha.head()

In [ ]:
#raw data
df_alpha.head().columns

In [ ]:
y_best1=np.array([gamma*A*np.exp(-gamma*A*t) for A in x])  
y_best2=np.array([gamma*A*np.exp(-beta*A*t) for A in x])  

label1=r'$\gamma A e^{-\gamma A t}$'
label2=r'$\gamma A e^{-\beta A t}$'
#plt.plot(x,y_best1,label=rf'$\gamma$')
plt.plot(x,y_best1,label=label1)
plt.plot(x,y_best2,label=label2)
plt.axvline(x=source_activity,color='m',linestyle='--')
plt.legend()
plt.show()
